# Major League Baseball Data ETL

## University of Denver Data Analytics Bootcamp

### Team MLB Fans

### May 17, 2019
  
  
**Presenters:**
* Tatsuya Sakurai
* Matthew Stewart
---

## Purpose


The purpose of this project was to perform "extract, transform, and load" (ETL) on various data pulled from Major League Baseball data sources.  
  
We extracted our data from the data sources listed below and transformed the data through various uses of python, including the _requests_ and _BeautifulSoup_ libraries, as well as pandas dataframes.  Finally, we loaded our transformed data to multiple collections within a Mongo database.

---

## Data Sources

[Major League Baseball Official Website](https://www.mlb.com/)  
[Major League Baseball API](https://appac.github.io/mlb-data-api-docs/)

---

## Data Extracted

From our data sources, we extracted the following:

* **MLB API**
 * List of all teams, including various pieces of team information
 * Colorado Rockies 40-man roster
 * Current hitting leaders (by batting average)
 * Career hitting stats for same hitting leaders
 * Current pitching leaders (by earned run average)
 * Career pitching stats for same pitching leaders
  
  
* **MLB Official Website:**
 * Latest news headlines and URLs
 * Most recent Colorado Rockies news headline and URL
 * Most recent Colorado Rockies video URL
 * Video URLs for above hitting leaders
 * Table data for current standings

---

### Initial Setup

In this section, we will import our dependencies and also establish the base URL for our API requests.

In [3]:
# Dependencies
import json
import requests
from pprint import pprint
import pandas as pd
import pymongo
from bs4 import BeautifulSoup
from splinter import Browser
import time

In [4]:
# Base URL for API requests
base_url = 'http://lookup-service-prod.mlb.com'

---

## MLB API Requests

---

### All Teams plus Team Facts

In this section, we will extract the names of all 30 teams, including various pieces of information about these teams.  We will ultimately store this information in a dictionary for future upload to our Mongo database.

In [5]:
# API endpoint
all_teams_endpoint = "/json/named.team_all_season.bam?sport_code='mlb'&all_star_sw='N'&sort_order=name_asc&season='2019'"

In [6]:
# Use requests.get to make API request
all_teams_response = requests.get(base_url + all_teams_endpoint)
print(all_teams_response)
print(all_teams_response.url)

<Response [200]>
http://lookup-service-prod.mlb.com/json/named.team_all_season.bam?sport_code='mlb'&all_star_sw='N'&sort_order=name_asc&season='2019'


In [7]:
# Convert response data to python object and print the results.
# From the results, we can see that we need to drill down into the "row" element.
all_teams_data = all_teams_response.json()
pprint(all_teams_data)

{'team_all_season': {'copyRight': ' Copyright 2019 MLB Advanced Media, L.P.  '
                                  'Use of any content on this page '
                                  'acknowledges agreement to the terms posted '
                                  'here '
                                  'http://gdx.mlb.com/components/copyright.txt  ',
                     'queryResults': {'created': '2019-05-18T06:05:11',
                                      'row': [{'active_sw': 'Y',
                                               'address': '401 East Jefferson '
                                                          'Street&#xa;Phoenix, '
                                                          'AZ&#x9;85004',
                                               'address_city': 'Phoenix',
                                               'address_country': '',
                                               'address_intl': 'N',
                                               'address_line1':

                                               'division_abbrev': 'ALW',
                                               'division_full': 'American '
                                                                'League West',
                                               'division_id': '200',
                                               'file_code': 'hou',
                                               'first_year_of_play': '1962',
                                               'franchise_code': 'HOU',
                                               'home_opener': '2019-04-05T00:00:00',
                                               'home_opener_time': '8:10:00 PM',
                                               'last_year_of_play': '2019',
                                               'league': 'AL',
                                               'league_abbrev': 'AL',
                                               'league_full': 'American League',
                               

                                                                     'League',
                                               'spring_league_id': '114',
                                               'state': 'CA',
                                               'store_url': 'losangeles.dodgers.mlb.com',
                                               'team_code': 'lan',
                                               'team_id': '119',
                                               'time_zone': 'PT',
                                               'time_zone_alt': 'America/Los_Angeles',
                                               'time_zone_generic': 'PT',
                                               'time_zone_num': '-7',
                                               'time_zone_text': 'PDT',
                                               'venue_id': '22',
                                               'venue_name': 'Dodger Stadium',
                                               've

                                               'website_url': 'phillies.com'},
                                              {'active_sw': 'Y',
                                               'address': '115 Federal '
                                                          'Street&#xa;Pittsburgh, '
                                                          'PA&#x9;15212',
                                               'address_city': 'Pittsburgh',
                                               'address_country': '',
                                               'address_intl': 'N',
                                               'address_line1': '115 Federal '
                                                                'Street',
                                               'address_line2': '',
                                               'address_line3': '',
                                               'address_province': '',
                                               'ad

                                               'name_display_full': 'Seattle '
                                                                    'Mariners',
                                               'name_display_long': 'Seattle '
                                                                    'Mariners',
                                               'name_display_short': 'Seattle',
                                               'name_short': 'Seattle',
                                               'phone_number': '(206) 346-4000',
                                               'season': '2019',
                                               'sport_code': 'mlb',
                                               'sport_code_display': 'Major '
                                                                     'League '
                                                                     'Baseball',
                                               'sport_code_name': 'MLB',
     

In [8]:
# Limit results to the "row" element
all_teams_data_for_df = all_teams_data['team_all_season']['queryResults']['row']

In [9]:
# Convert to dataframe
teams_df = pd.DataFrame(all_teams_data_for_df)
teams_df.head()

,active_sw,address,address_city,address_country,address_intl,address_line1,address_line2,address_line3,address_province,address_state,...,team_id,time_zone,time_zone_alt,time_zone_generic,time_zone_num,time_zone_text,venue_id,venue_name,venue_short,website_url
0,Y,"401 East Jefferson Street&#xa;Phoenix, AZ&#x9;...",Phoenix,,N,401 East Jefferson Street,,,,AZ,...,109,MST,America/Phoenix,MST,-7,MST,15,Chase Field,Chase Field,dbacks.com
1,Y,"755 Battery Avenue&#xa;Atlanta, GA&#x9;30339",Atlanta,USA,N,755 Battery Avenue,,,,GA,...,144,ET,America/New_York,ET,-4,EDT,4705,SunTrust Park,SunTrust Park,braves.com
2,Y,"333 West Camden Street&#xa;Baltimore, MD&#x9;2...",Baltimore,,N,333 West Camden Street,,,,MD,...,110,ET,America/New_York,ET,-4,EDT,2,Oriole Park at Camden Yards,Oriole Park,orioles.com
3,Y,"4 Yawkey Way&#xa;Boston, MA&#x9;2215",Boston,,N,4 Yawkey Way,,,,MA,...,111,ET,America/New_York,ET,-4,EDT,3,Fenway Park,Fenway Park,redsox.com
4,Y,"1060 West Addison&#xa;Chicago, IL&#x9;60613-4397",Chicago,,N,1060 West Addison,,,,IL,...,112,CT,America/Chicago,CT,-5,CDT,17,Wrigley Field,Wrigley Field,cubs.com


In [12]:
# Data cleanup
# Will grab only the elements we want and will clean up the names
teams_df = teams_df[
    [
        'mlb_org_id',
        'mlb_org',
        'city',
        'state',
        'division_full',
        'venue_name',
        'base_url'
    ]
]

teams_df.rename(columns = {
    'mlb_org_id': 'team_id',
    'mlb_org': 'team_name',
    'division_full': 'division'
}, inplace=True)

# Sort by division and team name for use as indices
teams_df.sort_values(['division', 'team_name'], inplace=True)

In [13]:
# Use division and team name as multiindex
teams_df.set_index(['division', 'team_name'], inplace=True)
teams_df

team_id           city state  \
division                team_name                                            
American League Central Chicago White Sox         145        Chicago    IL   
                        Cleveland Indians         114      Cleveland    OH   
                        Detroit Tigers            116        Detroit    MI   
                        Kansas City Royals        118    Kansas City    MO   
                        Minnesota Twins           142    Minneapolis    MN   
American League East    Baltimore Orioles         110      Baltimore    MD   
                        Boston Red Sox            111         Boston    MA   
                        New York Yankees          147          Bronx    NY   
                        Tampa Bay Rays            139      Tampa Bay    FL   
                        Toronto Blue Jays         141        Toronto    ON   
American League West    Houston Astros            117        Houston    TX   
                        Los Angeles Angels        108        Anaheim    CA   
                        Oakland Athletics         133        Oakland    CA   
                        Seattle Mariners          136        Seattle    WA   
                        Texas Rangers             140      Arlington    TX   
National League Central Chicago Cubs              112        Chicago    IL   
                        Cincinnati Reds           113     Cincinnati    OH   
                        Milwaukee Brewers         158      Milwaukee    WI   
                        Pittsburgh Pirates        134     Pittsburgh    PA   
                        St. Louis Cardinals       138      St. Louis    MO   
National League East    Atlanta Braves            144        Atlanta    GA   
                        Miami Marlins             146          Miami    FL   
                        New York Mets             121       New York    NY   
                        Philadelphia Phillies     143   Philadelphia    PA   
                        Washington Nationals      120     Washington    DC   
National League West    Arizona Diamondbacks      109        Phoenix    AZ   
                        Colorado Rockies          115         Denver    CO   
                        Los Angeles Dodgers       119    Los Angeles    CA   
                        San Diego Padres          135      San Diego    CA   
                        San Francisco Giants      137  San Francisco    CA   

                                                                 venue_name  \
division                team_name                                             
American League Central Chicago White Sox             Guaranteed Rate Field   
                        Cleveland Indians                 Progressive Field   
                        Detroit Tigers                        Comerica Park   
                        Kansas City Royals                 Kauffman Stadium   
                        Minnesota Twins                        Target Field   
American League East    Baltimore Orioles       Oriole Park at Camden Yards   
                        Boston Red Sox                          Fenway Park   
                        New York Yankees                     Yankee Stadium   
                        Tampa Bay Rays                      Tropicana Field   
                        Toronto Blue Jays                     Rogers Centre   
American League West    Houston Astros                     Minute Maid Park   
                        Los Angeles Angels                    Angel Stadium   
                        Oakland Athletics                  Oakland Coliseum   
                        Seattle Mariners                      T-Mobile Park   
                        Texas Rangers          Globe Life Park in Arlington   
National League Central Chicago Cubs                          Wrigley Field   
                        Cincinnati Reds            Great American Ball Park   
                        Milwaukee Brewers       

In [48]:
# Need to transpose the above dataframe for future conversion to dictionary
teams_df_transposed = teams_df.T
teams_df_transposed

division   American League Central                                     \
team_name        Chicago White Sox  Cleveland Indians  Detroit Tigers   
team_id                        145                114             116   
city                       Chicago          Cleveland         Detroit   
state                           IL                 OH              MI   
venue_name   Guaranteed Rate Field  Progressive Field   Comerica Park   
base_url          whitesox.mlb.com    indians.mlb.com  tigers.mlb.com   

division                                              American League East  \
team_name  Kansas City Royals Minnesota Twins            Baltimore Orioles   
team_id                   118             142                          110   
city              Kansas City     Minneapolis                    Baltimore   
state                      MO              MN                           MD   
venue_name   Kauffman Stadium    Target Field  Oriole Park at Camden Yards   
base_url       royals.mlb.com   twins.mlb.com              orioles.mlb.com   

division                                                      \
team_name   Boston Red Sox New York Yankees   Tampa Bay Rays   
team_id                111              147              139   
city                Boston            Bronx        Tampa Bay   
state                   MA               NY               FL   
venue_name     Fenway Park   Yankee Stadium  Tropicana Field   
base_url    redsox.mlb.com  yankees.mlb.com     rays.mlb.com   

division                             ...          National League East  \
team_name  Toronto Blue Jays         ...                Atlanta Braves   
team_id                  141         ...                           144   
city                 Toronto         ...                       Atlanta   
state                     ON         ...                            GA   
venue_name     Rogers Centre         ...                 SunTrust Park   
base_url    bluejays.mlb.com         ...                braves.mlb.com   

division                                                         \
team_name     Miami Marlins New York Mets Philadelphia Phillies   
team_id                 146           121                   143   
city                  Miami      New York          Philadelphia   
state                    FL            NY                    PA   
venue_name     Marlins Park    Citi Field    Citizens Bank Park   
base_url    marlins.mlb.com  mets.mlb.com      phillies.mlb.com   

division                        National League West                   \
team_name  Washington Nationals Arizona Diamondbacks Colorado Rockies   
team_id                     120                  109              115   
city                 Washington              Phoenix           Denver   
state                        DC                   AZ               CO   
venue_name       Nationals Park          Chase Field      Coors Field   
base_url      nationals.mlb.com       dbacks.mlb.com  rockies.mlb.com   

division                                                              
team_name  Los Angeles Dodgers San Diego Padres San Francisco Giants  
team_id                    119              135                  137  
city               Los Angeles        San Diego        San Francisco  
state                       CA               CA                   CA  
venue_name      Dodger Stadium       Petco Park          Oracle Park  
base_url       dodgers.mlb.com   padres.mlb.com       giants.mlb.com  

[5 rows x 30 columns]

In [49]:
# After transposition above, need to drop topmost level "division"
# Will use both dataframes in for loop below to create dictionary
teams_df_transposed_2 = teams_df_transposed.copy()
teams_df_transposed_2.columns = teams_df_transposed_2.columns.droplevel()
teams_df_transposed_2

team_name,Chicago White Sox,Cleveland Indians,Detroit Tigers,Kansas City Royals,Minnesota Twins,Baltimore Orioles,Boston Red Sox,New York Yankees,Tampa Bay Rays,Toronto Blue Jays,...,Atlanta Braves,Miami Marlins,New York Mets,Philadelphia Phillies,Washington Nationals,Arizona Diamondbacks,Colorado Rockies,Los Angeles Dodgers,San Diego Padres,San Francisco Giants
team_id,145,114,116,118,142,110,111,147,139,141,...,144,146,121,143,120,109,115,119,135,137
city,Chicago,Cleveland,Detroit,Kansas City,Minneapolis,Baltimore,Boston,Bronx,Tampa Bay,Toronto,...,Atlanta,Miami,New York,Philadelphia,Washington,Phoenix,Denver,Los Angeles,San Diego,San Francisco
state,IL,OH,MI,MO,MN,MD,MA,NY,FL,ON,...,GA,FL,NY,PA,DC,AZ,CO,CA,CA,CA
venue_name,Guaranteed Rate Field,Progressive Field,Comerica Park,Kauffman Stadium,Target Field,Oriole Park at Camden Yards,Fenway Park,Yankee Stadium,Tropicana Field,Rogers Centre,...,SunTrust Park,Marlins Park,Citi Field,Citizens Bank Park,Nationals Park,Chase Field,Coors Field,Dodger Stadium,Petco Park,Oracle Park
base_url,whitesox.mlb.com,indians.mlb.com,tigers.mlb.com,royals.mlb.com,twins.mlb.com,orioles.mlb.com,redsox.mlb.com,yankees.mlb.com,rays.mlb.com,bluejays.mlb.com,...,braves.mlb.com,marlins.mlb.com,mets.mlb.com,phillies.mlb.com,nationals.mlb.com,dbacks.mlb.com,rockies.mlb.com,dodgers.mlb.com,padres.mlb.com,giants.mlb.com


In [50]:
# Iterate through above dataframes to convert results to dictionary
# Final dictionary will be a dictionary of a list of dictionaries

# Begin with empty dictionary
teams_dict = {}

# Initialize list (for list of dictionaries) with the first column of data from df_2
teams_dict_values = [pd.DataFrame(teams_df_transposed_2.iloc[:,0]).to_dict('dict')]

# Iterate through remaining columns
for x in range(1, len(teams_df_transposed.columns)):
    
    # Look at top-most "division" layer in df_1.
    # If division (e.g., "American League Central") is the same from column to column,
    # continue to append to the list we initialized above.
    if teams_df_transposed.columns[x][0] == teams_df_transposed.columns[x-1][0]:
        teams_dict_values.append(pd.DataFrame(teams_df_transposed_2.iloc[:,x]).to_dict('dict'))
    
    # If division changes, add the list to the teams_dict dictionary
    # and re-initialize the list at the next column.
    else:
        teams_dict[teams_df_transposed.columns[x-1][0]] = teams_dict_values
        teams_dict_values = [pd.DataFrame(teams_df_transposed_2.iloc[:,x]).to_dict('dict')]

# After the for loop has finished, we have one more full list that has not yet
# been added to the dictionary.  Will add here.
teams_dict[teams_df_transposed.columns[len(teams_df_transposed.columns)-1][0]] = teams_dict_values

In [51]:
# Cannot upload the '.' character to Mongo.
# Need to remove '.' from 'St. Louis Cardinals'
teams_dict['National League Central'][4]['St Louis Cardinals'] = \
    teams_dict['National League Central'][4].pop('St. Louis Cardinals')
pprint(teams_dict)

{'American League Central': [{'Chicago White Sox': {'base_url': 'whitesox.mlb.com',
                                                    'city': 'Chicago',
                                                    'state': 'IL',
                                                    'team_id': '145',
                                                    'venue_name': 'Guaranteed '
                                                                  'Rate '
                                                                  'Field'}},
                             {'Cleveland Indians': {'base_url': 'indians.mlb.com',
                                                    'city': 'Cleveland',
                                                    'state': 'OH',
                                                    'team_id': '114',
                                                    'venue_name': 'Progressive '
                                                                  'Field'}},
                          

---

### Colorado Rockies 40-man Roster

In this section, we will extract the 40-man roster for the Colorado Rockies, in addition to information about each player.

In [52]:
# Iterate through the above teams dictionary to find the team ID 
# for the Rockies (team ID is 115)
for team in teams_dict['National League West']:
    for key, value in team.items():
        if key == 'Colorado Rockies':
            rox_id = team[key]['team_id']

rox_id

'115'

In [53]:
# Rockies roster endpoint and API request
rox_roster_endpoint = f"/json/named.roster_40.bam?team_id={rox_id}"
rox_roster_response = requests.get(base_url + rox_roster_endpoint)
print(rox_roster_response)
print(rox_roster_response.url)

<Response [200]>
http://lookup-service-prod.mlb.com/json/named.roster_40.bam?team_id=115


In [54]:
# Resulting roster data
rox_roster_data = rox_roster_response.json()
pprint(rox_roster_data)

{'roster_40': {'copyRight': ' Copyright 2019 MLB Advanced Media, L.P.  Use of '
                            'any content on this page acknowledges agreement '
                            'to the terms posted here '
                            'http://gdx.mlb.com/components/copyright.txt  ',
               'queryResults': {'created': '2019-05-18T06:00:35',
                                'row': [{'bats': 'S',
                                         'birth_date': '1994-06-04T00:00:00',
                                         'college': '',
                                         'end_date': '',
                                         'height_feet': '6',
                                         'height_inches': '5',
                                         'jersey_number': '62',
                                         'name_display_first_last': 'Yency '
                                                                    'Almonte',
                                         'name_displa

                                         'primary_position': '4',
                                         'pro_debut_date': '2017-08-12T00:00:00',
                                         'start_date': '2017-08-11T00:00:00',
                                         'starter_sw': 'N',
                                         'status_code': 'A',
                                         'team_abbrev': 'COL',
                                         'team_code': 'col',
                                         'team_id': '115',
                                         'team_name': 'Colorado Rockies',
                                         'throws': 'R',
                                         'weight': '208'},
                                        {'bats': 'L',
                                         'birth_date': '1985-04-01T00:00:00',
                                         'college': 'Jacksonville',
                                         'end_date': '',
                        

In [55]:
# Drill down to "row" element (as above)
rox_roster_data_for_df = rox_roster_data['roster_40']['queryResults']['row']

In [56]:
# Convert to dataframe
rox_roster_df = pd.DataFrame(rox_roster_data_for_df)
rox_roster_df.head()

,bats,birth_date,college,end_date,height_feet,height_inches,jersey_number,name_display_first_last,name_display_last_first,name_first,...,pro_debut_date,start_date,starter_sw,status_code,team_abbrev,team_code,team_id,team_name,throws,weight
0,S,1994-06-04T00:00:00,,,6,5,62,Yency Almonte,"Almonte, Yency",Yency,...,2018-06-21T00:00:00,2016-11-18T00:00:00,N,A,COL,col,115,Colorado Rockies,R,217
1,L,1989-12-30T00:00:00,Oregon,,6,3,44,Tyler Anderson,"Anderson, Tyler",Tyler,...,2016-06-12T00:00:00,2014-11-20T00:00:00,N,RM,COL,col,115,Colorado Rockies,L,215
2,R,1991-04-16T00:00:00,,,6,2,28,Nolan Arenado,"Arenado, Nolan",Nolan,...,2013-04-28T00:00:00,2013-04-28T00:00:00,N,A,COL,col,115,Colorado Rockies,R,215
3,R,1989-04-26T00:00:00,Texas Tech,,6,0,35,Chad Bettis,"Bettis, Chad",Chad,...,2013-08-01T00:00:00,2013-08-01T00:00:00,N,A,COL,col,115,Colorado Rockies,R,201
4,L,1986-07-01T00:00:00,Georgia Tech,,6,3,19,Charlie Blackmon,"Blackmon, Charlie",Charles,...,2011-06-07T00:00:00,2011-06-07T00:00:00,N,A,COL,col,115,Colorado Rockies,L,220


In [57]:
# Cleanup and sorting (as above)
# Will use 'position' and 'name' for future indices
rox_roster_df = rox_roster_df[
    [
        'name_display_last_first',
        'player_id',
        'jersey_number',
        'birth_date',
        'college',
        'pro_debut_date',
        'position_txt',
        'bats',
        'throws'
    ]
]

rox_roster_df.rename(columns = {
    'name_display_last_first': 'name',
    'position_txt': 'position',
}, inplace=True)

rox_roster_df.sort_values(['position', 'name'], inplace=True)

In [58]:
# Index setting, with 'postion' and 'name'
rox_roster_df.set_index(['position', 'name'], inplace=True)
rox_roster_df.head()

player_id jersey_number           birth_date  \
position name                                                            
1B       Murphy, Daniel      502517             9  1985-04-01T00:00:00   
         Reynolds, Mark      448602            12  1983-08-03T00:00:00   
2B       Hampson, Garrett    641658             1  1994-10-10T00:00:00   
         McMahon, Ryan       641857            24  1994-12-14T00:00:00   
3B       Arenado, Nolan      571448            28  1991-04-16T00:00:00   

                                    college       pro_debut_date bats throws  
position name                                                                 
1B       Murphy, Daniel        Jacksonville  2008-08-02T00:00:00    L      R  
         Reynolds, Mark            Virginia  2007-05-16T00:00:00    R      R  
2B       Hampson, Garrett  Long Beach State  2018-07-21T00:00:00    R      R  
         McMahon, Ryan                       2017-08-12T00:00:00    L      R  
3B       Arenado, Nolan                      2013-04-28T00:00:00    R      R

In [59]:
# Cleanup date columns to eliminate the time information using lambda functions
rox_roster_df['birth_date'] = rox_roster_df['birth_date'].apply(lambda x: x.split('T')[0])
rox_roster_df['pro_debut_date'] = rox_roster_df['pro_debut_date'].apply(lambda x: x.split('T')[0])

rox_roster_df

player_id jersey_number  birth_date  \
position name                                                     
1B       Murphy, Daniel        502517             9  1985-04-01   
         Reynolds, Mark        448602            12  1983-08-03   
2B       Hampson, Garrett      641658             1  1994-10-10   
         McMahon, Ryan         641857            24  1994-12-14   
3B       Arenado, Nolan        571448            28  1991-04-16   
         Fuentes, Josh         658069             8  1993-02-19   
         Valaika, Pat          642162             4  1992-09-09   
C        Iannetta, Chris       455104            22  1983-04-08   
         Wolters, Tony         547172            14  1992-06-09   
CF       Daza, Yonathan        602074            31  1994-02-28   
         Desmond, Ian          435622            20  1985-09-20   
         Tapia, Raimel         606132            15  1994-02-04   
LF       Dahl, David           621311            26  1994-04-01   
P        Almonte, Yency        622075            62  1994-06-04   
         Anderson, Tyler       542881            44  1989-12-30   
         Bettis, Chad          518452            35  1989-04-26   
         Castellani, Ryan      624418            60  1996-04-01   
         Davis, Wade           451584            71  1985-09-07   
         Dunn, Mike            445197            38  1985-05-23   
         Estevez, Carlos       608032            54  1992-12-28   
         Freeland, Kyle        607536            21  1993-05-14   
         Gray, Jon             592351            55  1991-11-05   
         Hoffman, Jeff         656546            34  1993-01-08   
         Johnson, DJ           597113            63  1989-08-30   
         Lawrence, Justin      664875            74  1994-11-25   
         Marquez, German       608566            48  1995-02-22   
         McGee, Jake           459429            51  1986-08-06   
         Musgrave, Harrison    643466            59  1992-03-03   
         Oberg, Scott          623184            45  1990-03-13   
         Oh, Seunghwan         493200            18  1982-07-15   
         Pazos, James          572021                1991-05-05   
         Rusin, Chris          543734            52  1986-10-22   
         Senzatela, Antonio    622608            49  1995-01-21   
         Shaw, Bryan           543766            29  1987-11-08   
         Tinoco, Jesus         622786            32  1995-04-30   
RF       Blackmon, Charlie     453568            19  1986-07-01   
         Cuevas, Noel          571595            56  1991-10-02   
         Hilliard, Sam         656541            43  1994-02-21   
SS       Rodgers, Brendan      663898             7  1996-08-09   
         Story, Trevor         596115            27  1992-11-15   

                                                             college  \
position name                                                          
1B       Murphy, Daniel                                 Jacksonville   
         Reynolds, Mark                                     Virginia   
2B       Hampson, Garrett                           Long Beach State   
         McMahon, Ryan                                                 
3B       Arenado, Nolan                                                
         Fuentes, Josh                              Missouri Baptist   
         Valaika, Pat                                           UCLA   
C        Iannetta, Chris                              North Carolina   
         Wolters, Tony                                                 
CF       Daza, Yonathan                                                
         Desmond, Ian                                                  
         Tapia, Raimel                                                 
LF       Dahl, David                                                   
P        Almonte, Yency                                                
         Anderson, Tyler                                     

In [62]:
# Follow same transposition and for loop methodology as "teams" section above.
# Will not comment further on this section as it follows the same methodololgy.
rox_roster_df_transposed = rox_roster_df.T
rox_roster_df_transposed

position                   1B                               2B                \
name           Murphy, Daniel Reynolds, Mark  Hampson, Garrett McMahon, Ryan   
player_id              502517         448602            641658        641857   
jersey_number               9             12                 1            24   
birth_date         1985-04-01     1983-08-03        1994-10-10    1994-12-14   
college          Jacksonville       Virginia  Long Beach State                 
pro_debut_date     2008-08-02     2007-05-16        2018-07-21    2017-08-12   
bats                        L              R                 R             L   
throws                      R              R                 R             R   

position                   3B                                              C  \
name           Arenado, Nolan     Fuentes, Josh Valaika, Pat Iannetta, Chris   
player_id              571448            658069       642162          455104   
jersey_number              28                 8            4              22   
birth_date         1991-04-16        1993-02-19   1992-09-09      1983-04-08   
college                        Missouri Baptist         UCLA  North Carolina   
pro_debut_date     2013-04-28        2019-04-06   2016-09-06      2006-08-27   
bats                        R                 R            R               R   
throws                      R                 R            R               R   

position                                 CF      ...                 P  \
name           Wolters, Tony Daza, Yonathan      ...      Pazos, James   
player_id             547172         602074      ...            572021   
jersey_number             14             31      ...                     
birth_date        1992-06-09     1994-02-28      ...        1991-05-05   
college                                          ...         San Diego   
pro_debut_date    2016-04-05     2019-04-09      ...        2015-09-05   
bats                       L              R      ...                 R   
throws                     R              R      ...                 L   

position                                                          \
name           Rusin, Chris Senzatela, Antonio       Shaw, Bryan   
player_id            543734             622608            543766   
jersey_number            52                 49                29   
birth_date       1986-10-22         1995-01-21        1987-11-08   
college            Kentucky                     Long Beach State   
pro_debut_date   2012-08-21         2017-04-06        2011-06-10   
bats                      L                  R                 S   
throws                    L                  R                 R   

position                                    RF  \
name           Tinoco, Jesus Blackmon, Charlie   
player_id             622786            453568   
jersey_number             32                19   
birth_date        1995-04-30        1986-07-01   
college                           Georgia Tech   
pro_debut_date                      2011-06-07   
bats                       R                 L   
throws                     R                 L   

position                                                                \
name                                       Cuevas, Noel  Hilliard, Sam   
player_id                                        571595         656541   
jersey_number                                        56             43   
birth_date                                   1991-10-02     1994-02-21   
college         Interamerican University of Puerto Rico  Wichita State   
pro_debut_date                               2018-04-22                  
bats                                                  R              L   
throws                                                R              L   

position                     SS                
name           Rodgers, Brendan Story, Trevor  
player_id                663898        596115  
jersey_number   

In [63]:
rox_roster_df_transposed_2 = rox_roster_df_transposed.copy()
rox_roster_df_transposed_2.columns = rox_roster_df_transposed_2.columns.droplevel()
rox_roster_df_transposed_2

name,"Murphy, Daniel","Reynolds, Mark","Hampson, Garrett","McMahon, Ryan","Arenado, Nolan","Fuentes, Josh","Valaika, Pat","Iannetta, Chris","Wolters, Tony","Daza, Yonathan",...,"Pazos, James","Rusin, Chris","Senzatela, Antonio","Shaw, Bryan","Tinoco, Jesus","Blackmon, Charlie","Cuevas, Noel","Hilliard, Sam","Rodgers, Brendan","Story, Trevor"
player_id,502517,448602,641658,641857,571448,658069,642162,455104,547172,602074,...,572021,543734,622608,543766,622786,453568,571595,656541,663898,596115
jersey_number,9,12,1,24,28,8,4,22,14,31,...,,52,49,29,32,19,56,43,7,27
birth_date,1985-04-01,1983-08-03,1994-10-10,1994-12-14,1991-04-16,1993-02-19,1992-09-09,1983-04-08,1992-06-09,1994-02-28,...,1991-05-05,1986-10-22,1995-01-21,1987-11-08,1995-04-30,1986-07-01,1991-10-02,1994-02-21,1996-08-09,1992-11-15
college,Jacksonville,Virginia,Long Beach State,,,Missouri Baptist,UCLA,North Carolina,,,...,San Diego,Kentucky,,Long Beach State,,Georgia Tech,Interamerican University of Puerto Rico,Wichita State,,
pro_debut_date,2008-08-02,2007-05-16,2018-07-21,2017-08-12,2013-04-28,2019-04-06,2016-09-06,2006-08-27,2016-04-05,2019-04-09,...,2015-09-05,2012-08-21,2017-04-06,2011-06-10,,2011-06-07,2018-04-22,,2019-05-17,2016-04-04
bats,L,R,R,L,R,R,R,R,L,R,...,R,L,R,S,R,L,R,L,R,R
throws,R,R,R,R,R,R,R,R,R,R,...,L,L,R,R,R,L,R,L,R,R


In [64]:
rox_roster_dict = {}
rox_roster_dict_values = [pd.DataFrame(rox_roster_df_transposed_2.iloc[:,0]).to_dict('dict')]
for x in range(1, len(rox_roster_df_transposed.columns)):
    if rox_roster_df_transposed.columns[x][0] == rox_roster_df_transposed.columns[x-1][0]:
        rox_roster_dict_values.append(pd.DataFrame(rox_roster_df_transposed_2.iloc[:,x]).to_dict('dict'))
    else:
        rox_roster_dict[rox_roster_df_transposed.columns[x-1][0]] = rox_roster_dict_values
        rox_roster_dict_values = [pd.DataFrame(rox_roster_df_transposed_2.iloc[:,x]).to_dict('dict')]
rox_roster_dict[rox_roster_df_transposed.columns[len(rox_roster_df_transposed.columns)-1][0]] = rox_roster_dict_values
pprint(rox_roster_dict)

{'1B': [{'Murphy, Daniel': {'bats': 'L',
                            'birth_date': '1985-04-01',
                            'college': 'Jacksonville',
                            'jersey_number': '9',
                            'player_id': '502517',
                            'pro_debut_date': '2008-08-02',
                            'throws': 'R'}},
        {'Reynolds, Mark': {'bats': 'R',
                            'birth_date': '1983-08-03',
                            'college': 'Virginia',
                            'jersey_number': '12',
                            'player_id': '448602',
                            'pro_debut_date': '2007-05-16',
                            'throws': 'R'}}],
 '2B': [{'Hampson, Garrett': {'bats': 'R',
                              'birth_date': '1994-10-10',
                              'college': 'Long Beach State',
                              'jersey_number': '1',
                              'player_id': '641658',
                   

---

### Current Hitting Leaders

In this section, we will extract information on the current hitting leaders.  We will extract the top five leaders by batting average.  In case of a tie, the API will return all tied hitters, resulting in more than five names.

In [65]:
# endpoint and API request
hitting_leader_endpoint = f"/json/named.leader_hitting_repeater.bam?sport_code='mlb'&results=5&game_type='R'&season='2019'&sort_column='avg'"
hitting_leader_response = requests.get(base_url + hitting_leader_endpoint)
print(hitting_leader_response)
print(hitting_leader_response.url)

<Response [200]>
http://lookup-service-prod.mlb.com/json/named.leader_hitting_repeater.bam?sport_code='mlb'&results=5&game_type='R'&season='2019'&sort_column='avg'


In [66]:
# python object
hitting_leader_data = hitting_leader_response.json()
pprint(hitting_leader_data)

{'leader_hitting_repeater': {'copyRight': ' Copyright 2019 MLB Advanced Media, '
                                          'L.P.  Use of any content on this '
                                          'page acknowledges agreement to the '
                                          'terms posted here '
                                          'http://gdx.mlb.com/components/copyright.txt  ',
                             'leader_hitting_mux': {'queryResults': {'created': '2019-05-18T06:26:00',
                                                                     'row': [{'ab': '156',
                                                                              'ao': '44',
                                                                              'avg': '.404',
                                                                              'bats': 'L',
                                                                              'bb': '25',
                                                 

In [67]:
# "row" element
hitting_leader_data_for_df = hitting_leader_data['leader_hitting_repeater']['leader_hitting_mux']['queryResults']['row']

In [68]:
# dataframe
hitting_leader_df = pd.DataFrame(hitting_leader_data_for_df)
hitting_leader_df

,ab,ao,avg,bats,bb,cs,d,g,gdp,gidp,...,sport_id,t,tb,team,team_abbrev,team_brief,team_id,team_name,tpa,xbh
0,156,44,.404,L,25,3,10,44,2,2,...,1,1,123,lan,LAD,Dodgers,119,Los Angeles Dodgers,185,27
1,148,42,.351,L,15,4,11,42,2,2,...,1,1,71,nyn,NYM,Mets,121,New York Mets,171,14
2,125,33,.344,S,8,0,8,40,1,1,...,1,0,60,pit,PIT,Pirates,134,Pittsburgh Pirates,133,11
3,131,30,.336,R,11,0,8,45,4,4,...,1,0,61,sln,STL,Cardinals,138,St. Louis Cardinals,144,11
4,149,33,.336,L,29,0,7,42,4,4,...,1,0,111,mil,MIL,Brewers,158,Milwaukee Brewers,182,25


In [69]:
# column extraction and name cleanup
hitting_leader_df = hitting_leader_df[
    [
        'name_display_last_first',
        'player_id',
        'team_name',
        'team_id',
        'bats',
        'pos',
        'avg',
        'slg',
        'obp',
        'ops',
        'rbi',
        'h',
        'ab',
        'hr',
        'bb',
        'sac',
        'hbp',
        'so',
        'sb'
    ]
]

hitting_leader_df.rename(columns = {
    'name_display_last_first': 'name',
    'pos': 'position',
    'avg': 'batting_avg',
    'slg': 'slugging_pct',
    'obp': 'on_base_pct',
    'ops': 'on_base_plus_slugging',
    'h': 'hits',
    'ab': 'at_bats',
    'hr': 'homeruns',
    'bb': 'walks',
    'sac': 'sacrifice_fly',
    'hbp': 'hit_by_pitch',
    'so': 'strikeouts',
    'sb': 'stolen_bases'
}, inplace=True)

hitting_leader_df

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,name,player_id,team_name,team_id,bats,position,batting_avg,slugging_pct,on_base_pct,on_base_plus_slugging,rbi,hits,at_bats,homeruns,walks,sacrifice_fly,hit_by_pitch,strikeouts,stolen_bases
0,"Bellinger, Cody",641355,Los Angeles Dodgers,119,L,1B,.404,.788,.481,1.270,42,63,156,16,25,0,1,26,7
1,"McNeil, Jeff",643446,New York Mets,121,L,2B,.351,.480,.439,.918,14,52,148,2,15,0,8,19,0
2,"Cabrera, Melky",466320,Pittsburgh Pirates,134,S,OF,.344,.480,.383,.863,13,43,125,3,8,0,0,15,1
3,"Martinez, Jose",500874,St. Louis Cardinals,138,R,OF,.336,.466,.389,.855,20,44,131,3,11,0,1,26,1
4,"Yelich, Christian",592885,Milwaukee Brewers,158,L,LF,.336,.745,.451,1.196,40,50,149,18,29,0,3,32,9


In [70]:
# Rather than transposing, as we did above, we will convert the above dataframe directly to a dictionary.
# This will result in a list of dictionaries.
# Below, when we upload this to our Mongo dictionary, we will end up with one document per player.
hitting_leader_dict = hitting_leader_df.to_dict(orient='records')
pprint(hitting_leader_dict)

[{'at_bats': '156',
  'bats': 'L',
  'batting_avg': '.404',
  'hit_by_pitch': '1',
  'hits': '63',
  'homeruns': '16',
  'name': 'Bellinger, Cody',
  'on_base_pct': '.481',
  'on_base_plus_slugging': '1.270',
  'player_id': '641355',
  'position': '1B',
  'rbi': '42',
  'sacrifice_fly': '0',
  'slugging_pct': '.788',
  'stolen_bases': '7',
  'strikeouts': '26',
  'team_id': '119',
  'team_name': 'Los Angeles Dodgers',
  'walks': '25'},
 {'at_bats': '148',
  'bats': 'L',
  'batting_avg': '.351',
  'hit_by_pitch': '8',
  'hits': '52',
  'homeruns': '2',
  'name': 'McNeil, Jeff',
  'on_base_pct': '.439',
  'on_base_plus_slugging': '.918',
  'player_id': '643446',
  'position': '2B',
  'rbi': '14',
  'sacrifice_fly': '0',
  'slugging_pct': '.480',
  'stolen_bases': '0',
  'strikeouts': '19',
  'team_id': '121',
  'team_name': 'New York Mets',
  'walks': '15'},
 {'at_bats': '125',
  'bats': 'S',
  'batting_avg': '.344',
  'hit_by_pitch': '0',
  'hits': '43',
  'homeruns': '3',
  'name': 'Ca

---

### Career Stats for Hitting Leaders

In this section, we will extract the career stats for the hitting leaders identified in the above section.

In [71]:
# extract the hitting leader player IDs from the hitting leader dataframe above
hitting_leader_idents = list(hitting_leader_df['player_id'])
hitting_leader_idents

['641355', '643446', '466320', '500874', '592885']

In [72]:
# iterate through the above IDs and make an API request for each ID.
# store results in dictionary for use below
career_hitting_dict = {}
for ident in hitting_leader_idents:
    career_hitting_endpoint = f"/json/named.sport_career_hitting.bam?league_list_id='mlb'&game_type='R'&player_id={ident}"
    career_hitting_response = requests.get(base_url + career_hitting_endpoint)
    career_hitting_data = career_hitting_response.json()['sport_career_hitting']['queryResults']['row']
    career_hitting_dict[ident] = career_hitting_data
pprint(career_hitting_dict)    

{'466320': {'ab': '6625',
            'ao': '1826',
            'avg': '.287',
            'babip': '.309',
            'bb': '501',
            'cs': '37',
            'd': '369',
            'g': '1794',
            'gidp': '168',
            'gidp_opp': '1180',
            'go': '2290',
            'go_ao': '1.25',
            'h': '1899',
            'hbp': '26',
            'hfly': '274',
            'hgnd': '729',
            'hldr': '892',
            'hpop': '4',
            'hr': '140',
            'ibb': '40',
            'lob': '2696',
            'np': '26793',
            'obp': '.336',
            'ops': '.755',
            'player_id': '466320',
            'ppa': '3.69',
            'r': '865',
            'rbi': '820',
            'roe': '87',
            'sac': '47',
            'sb': '100',
            'sf': '64',
            'slg': '.419',
            'so': '865',
            'sport': 'MLB',
            'sport_code': 'mlb',
            'sport_id': '1',
            '

In [73]:
# convert above dictionary to dataframe.
# use transposition and reset_index so that we can get the desired column headers (with no index)
career_hitting_df = pd.DataFrame(career_hitting_dict).T.reset_index(drop=True)
career_hitting_df

,ab,ao,avg,babip,bb,cs,d,g,gidp,gidp_opp,...,so,sport,sport_code,sport_id,t,tb,team_count,tpa,wo,xbh
0,1193,316,.282,.320,158,7,64,338,14,186,...,323,MLB,mlb,1,12,664,1,1365,1,156
1,373,116,.338,.372,29,5,22,105,4,72,...,43,MLB,mlb,1,7,177,1,419,0,34
2,6625,1826,.287,.309,501,37,369,1794,168,1180,...,865,MLB,mlb,1,44,2776,8,7263,16,553
3,953,216,.313,.360,94,3,52,315,28,173,...,191,MLB,mlb,1,1,454,1,1059,1,87
4,3201,489,.299,.357,397,22,187,832,77,505,...,746,MLB,mlb,1,21,1524,2,3645,6,321


In [74]:
# column extraction and name cleanup
career_hitting_df = career_hitting_df[
    [
        'player_id',
        'avg',
        'slg',
        'obp',
        'ops',
        'rbi',
        'h',
        'ab',
        'hr',
        'bb',
        'sac',
        'hbp',
        'so',
        'sb'
    ]
]

career_hitting_df.rename(columns = {
    'avg': 'career_battng_avg',
    'slg': 'career_slgng_pct',
    'obp': 'career_on_base_pct',
    'ops': 'career_on_base_plus_slg',
    'rbi': 'career_rbi',
    'h': 'career_hits',
    'ab': 'career_at_bats',
    'hr': 'career_homeruns',
    'bb': 'career_walks',
    'sac': 'career_sac_fly',
    'hbp': 'career_hit_by_pitch',
    'so': 'career_strikeouts',
    'sb': 'career_stolen_bases'
}, inplace=True)

career_hitting_df

,player_id,career_battng_avg,career_slgng_pct,career_on_base_pct,career_on_base_plus_slg,career_rbi,career_hits,career_at_bats,career_homeruns,career_walks,career_sac_fly,career_hit_by_pitch,career_strikeouts,career_stolen_bases
0,641355,.282,.557,.366,.922,215,336,1193,80,158,0,5,323,31
1,643446,.338,.475,.405,.879,33,126,373,5,29,4,13,43,7
2,466320,.287,.419,.336,.755,820,1899,6625,140,501,47,26,865,100
3,500874,.313,.476,.374,.850,150,298,953,34,94,3,3,191,5
4,592885,.299,.476,.379,.855,443,956,3201,113,397,3,26,746,103


In [75]:
# API request for careers stats only give player IDs, not names.
# Merge with the above hitting leader dataframe (from previous section) to get the name column
career_hitting_df_merge = career_hitting_df.merge(hitting_leader_df, on = 'player_id')
career_hitting_df_merge

,player_id,career_battng_avg,career_slgng_pct,career_on_base_pct,career_on_base_plus_slg,career_rbi,career_hits,career_at_bats,career_homeruns,career_walks,...,on_base_plus_slugging,rbi,hits,at_bats,homeruns,walks,sacrifice_fly,hit_by_pitch,strikeouts,stolen_bases
0,641355,.282,.557,.366,.922,215,336,1193,80,158,...,1.270,42,63,156,16,25,0,1,26,7
1,643446,.338,.475,.405,.879,33,126,373,5,29,...,.918,14,52,148,2,15,0,8,19,0
2,466320,.287,.419,.336,.755,820,1899,6625,140,501,...,.863,13,43,125,3,8,0,0,15,1
3,500874,.313,.476,.374,.850,150,298,953,34,94,...,.855,20,44,131,3,11,0,1,26,1
4,592885,.299,.476,.379,.855,443,956,3201,113,397,...,1.196,40,50,149,18,29,0,3,32,9


In [76]:
# final column cleanup
career_hitting_df_merge = career_hitting_df_merge[
    [
        'player_id',
        'name',
        'career_battng_avg',
        'career_slgng_pct',
        'career_on_base_pct',
        'career_on_base_plus_slg',
        'career_rbi',
        'career_hits',
        'career_at_bats',
        'career_homeruns',
        'career_walks',
        'career_sac_fly',
        'career_hit_by_pitch',
        'career_strikeouts',
        'career_stolen_bases'
    ]
]
career_hitting_df_merge

,player_id,name,career_battng_avg,career_slgng_pct,career_on_base_pct,career_on_base_plus_slg,career_rbi,career_hits,career_at_bats,career_homeruns,career_walks,career_sac_fly,career_hit_by_pitch,career_strikeouts,career_stolen_bases
0,641355,"Bellinger, Cody",.282,.557,.366,.922,215,336,1193,80,158,0,5,323,31
1,643446,"McNeil, Jeff",.338,.475,.405,.879,33,126,373,5,29,4,13,43,7
2,466320,"Cabrera, Melky",.287,.419,.336,.755,820,1899,6625,140,501,47,26,865,100
3,500874,"Martinez, Jose",.313,.476,.374,.850,150,298,953,34,94,3,3,191,5
4,592885,"Yelich, Christian",.299,.476,.379,.855,443,956,3201,113,397,3,26,746,103


In [77]:
# convert to list of dictionaries, same as previous sections
career_hitting_dict = career_hitting_df_merge.to_dict(orient='records')
pprint(career_hitting_dict)

[{'career_at_bats': '1193',
  'career_battng_avg': '.282',
  'career_hit_by_pitch': '5',
  'career_hits': '336',
  'career_homeruns': '80',
  'career_on_base_pct': '.366',
  'career_on_base_plus_slg': '.922',
  'career_rbi': '215',
  'career_sac_fly': '0',
  'career_slgng_pct': '.557',
  'career_stolen_bases': '31',
  'career_strikeouts': '323',
  'career_walks': '158',
  'name': 'Bellinger, Cody',
  'player_id': '641355'},
 {'career_at_bats': '373',
  'career_battng_avg': '.338',
  'career_hit_by_pitch': '13',
  'career_hits': '126',
  'career_homeruns': '5',
  'career_on_base_pct': '.405',
  'career_on_base_plus_slg': '.879',
  'career_rbi': '33',
  'career_sac_fly': '4',
  'career_slgng_pct': '.475',
  'career_stolen_bases': '7',
  'career_strikeouts': '43',
  'career_walks': '29',
  'name': 'McNeil, Jeff',
  'player_id': '643446'},
 {'career_at_bats': '6625',
  'career_battng_avg': '.287',
  'career_hit_by_pitch': '26',
  'career_hits': '1899',
  'career_homeruns': '140',
  'career

---

_**The remaining two API requests (pitching leaders and career stats for pitching leaders) follow the exact same methodology as the previous two sections.  Will not comment in these sections.  See previous sections if clarification needed.**_

---

### Current Pitching Leaders

In this section, we will extract information on the current pitching leaders.  We will extract the top five leaders by earned run average.  In case of a tie, the API will return all tied pitchers, resulting in more than five names.

In [78]:
pitching_leader_endpoint = f"/json/named.leader_pitching_repeater.bam?sport_code='mlb'&results=5&game_type='R'&season='2019'&sort_column='era'"
pitching_leader_response = requests.get(base_url + pitching_leader_endpoint)
print(pitching_leader_response)
print(pitching_leader_response.url)

<Response [200]>
http://lookup-service-prod.mlb.com/json/named.leader_pitching_repeater.bam?sport_code='mlb'&results=5&game_type='R'&season='2019'&sort_column='era'


In [79]:
pitching_leader_data = pitching_leader_response.json()
pprint(pitching_leader_data)

{'leader_pitching_repeater': {'copyRight': ' Copyright 2019 MLB Advanced '
                                           'Media, L.P.  Use of any content on '
                                           'this page acknowledges agreement '
                                           'to the terms posted here '
                                           'http://gdx.mlb.com/components/copyright.txt  ',
                              'leader_pitching_mux': {'queryResults': {'created': '2019-05-18T06:26:28',
                                                                       'row': [{'ab': '192',
                                                                                'ao': '57',
                                                                                'avg': '.240',
                                                                                'bb': '16',
                                                                                'bb_9': '2.73',
                             

In [80]:
pitching_leader_data_for_df = pitching_leader_data['leader_pitching_repeater']['leader_pitching_mux']['queryResults']['row']

In [81]:
pitching_leader_df = pd.DataFrame(pitching_leader_data_for_df)
pitching_leader_df

,ab,ao,avg,bb,bb_9,bk,cg,cs,er,era,...,tbf,team,team_abbrev,team_id,team_name,throws,w,whip,wp,wpct
0,192,57,.240,16,2.73,0,0,2,9,1.54,...,213,mil,MIL,158,Milwaukee Brewers,R,5,1.18,1,1.000
1,185,40,.189,3,0.52,0,1,0,10,1.72,...,189,lan,LAD,119,Los Angeles Dodgers,L,5,0.73,0,.833
2,173,39,.202,9,1.68,0,0,2,10,1.86,...,183,tba,TB,139,Tampa Bay Rays,R,6,0.91,0,.857
3,208,34,.163,25,3.65,2,0,2,13,1.90,...,239,cin,CIN,113,Cincinnati Reds,R,5,0.96,1,.833
4,161,41,.199,16,3.15,0,0,1,10,1.97,...,178,pit,PIT,134,Pittsburgh Pirates,R,4,1.05,2,.800


In [82]:
pitching_leader_df = pitching_leader_df[
    [
        'name_display_last_first',
        'player_id',
        'team_name',
        'team_id',
        'era',
        'ip',
        'avg',
        'h',
        'h_9',
        'hr',
        'bb',
        'bb_9',
        'so',
        'k_9',
        'k_bb'
    ]
]

pitching_leader_df.rename(columns = {
    'name_display_last_first': 'name',
    'ip': 'innings_pitched',
    'avg': 'opposing_bat_avg',
    'h': 'hits',
    'h_9': 'hits_per_9',
    'hr': 'homeruns',
    'bb': 'walks',
    'bb_9': 'walks_per_9',
    'so': 'strikeouts',
    'k_9': 'strikeouts_per_9',
    'k_bb': 'strikeouts_to_walks'
}, inplace=True)

pitching_leader_df

,name,player_id,team_name,team_id,era,innings_pitched,opposing_bat_avg,hits,hits_per_9,homeruns,walks,walks_per_9,strikeouts,strikeouts_per_9,strikeouts_to_walks
0,"Davies, Zach",605200,Milwaukee Brewers,158,1.54,52.2,.240,46,7.86,4,16,2.73,36,6.15,2.25
1,"Ryu, Hyun-Jin",547943,Los Angeles Dodgers,119,1.72,52.1,.189,35,6.02,6,3,0.52,54,9.29,18.00
2,"Glasnow, Tyler",607192,Tampa Bay Rays,139,1.86,48.1,.202,35,6.52,3,9,1.68,55,10.24,6.11
3,"Castillo, Luis",622491,Cincinnati Reds,113,1.90,61.2,.163,34,4.96,3,25,3.65,76,11.09,3.04
4,"Lyles, Jordan",543475,Pittsburgh Pirates,134,1.97,45.2,.199,32,6.31,4,16,3.15,47,9.26,2.94


In [83]:
pitching_leader_dict = pitching_leader_df.to_dict(orient='records')
pprint(pitching_leader_dict)

[{'era': '1.54',
  'hits': '46',
  'hits_per_9': '7.86',
  'homeruns': '4',
  'innings_pitched': '52.2',
  'name': 'Davies, Zach',
  'opposing_bat_avg': '.240',
  'player_id': '605200',
  'strikeouts': '36',
  'strikeouts_per_9': '6.15',
  'strikeouts_to_walks': '2.25',
  'team_id': '158',
  'team_name': 'Milwaukee Brewers',
  'walks': '16',
  'walks_per_9': '2.73'},
 {'era': '1.72',
  'hits': '35',
  'hits_per_9': '6.02',
  'homeruns': '6',
  'innings_pitched': '52.1',
  'name': 'Ryu, Hyun-Jin',
  'opposing_bat_avg': '.189',
  'player_id': '547943',
  'strikeouts': '54',
  'strikeouts_per_9': '9.29',
  'strikeouts_to_walks': '18.00',
  'team_id': '119',
  'team_name': 'Los Angeles Dodgers',
  'walks': '3',
  'walks_per_9': '0.52'},
 {'era': '1.86',
  'hits': '35',
  'hits_per_9': '6.52',
  'homeruns': '3',
  'innings_pitched': '48.1',
  'name': 'Glasnow, Tyler',
  'opposing_bat_avg': '.202',
  'player_id': '607192',
  'strikeouts': '55',
  'strikeouts_per_9': '10.24',
  'strikeouts_to

---

### Career Stats for Pitching Leaders

In this section, we will extract the career stats for the pitching leaders identified in the above section.

In [84]:
pitching_leader_idents = list(pitching_leader_df['player_id'])
pitching_leader_idents

['605200', '547943', '607192', '622491', '543475']

In [85]:
career_pitching_dict = {}
for ident in pitching_leader_idents:
    career_pitching_endpoint = f"/json/named.sport_career_pitching.bam?league_list_id='mlb'&game_type='R'&player_id={ident}"
    career_pitching_response = requests.get(base_url + career_pitching_endpoint)
    career_pitching_data = career_pitching_response.json()['sport_career_pitching']['queryResults']['row']
    career_pitching_dict[ident] = career_pitching_data
pprint(career_pitching_dict)    

{'543475': {'ab': '3222',
            'ao': '762',
            'avg': '.277',
            'babip': '.311',
            'bb': '276',
            'bb9': '3.05',
            'bk': '2',
            'bq': '104',
            'bqs': '49',
            'cg': '1',
            'cs': '12',
            'db': '182',
            'er': '461',
            'era': '5.09',
            'g': '225',
            'gf': '32',
            'gidp': '74',
            'gidp_opp': '533',
            'go': '1017',
            'go_ao': '1.33',
            'gs': '123',
            'h': '892',
            'h9': '9.86',
            'hb': '43',
            'hfly': '154',
            'hgnd': '320',
            'hld': '8',
            'hldr': '415',
            'hpop': '3',
            'hr': '101',
            'hr9': '1.12',
            'ibb': '15',
            'ip': '814.1',
            'ir': '54',
            'irs': '18',
            'k9': '6.60',
            'kbb': '2.16',
            'l': '53',
            'np': '13603',

In [86]:
career_pitching_df = pd.DataFrame(career_pitching_dict).T.reset_index(drop=True)
career_pitching_df

,ab,ao,avg,babip,bb,bb9,bk,bq,bqs,cg,...,sport_id,sv,svo,tbf,team_count,tr,w,whip,wp,wpct
0,1936,458,.263,.298,145,2.57,0,50,18,0,...,1,0,0,2131,1,17,38,1.29,7,.603
1,2315,515,.248,.300,143,2.11,1,38,11,3,...,1,1,1,2492,1,9,45,1.17,11,.608
2,925,190,.245,.311,119,4.37,3,38,13,0,...,1,0,0,1061,2,6,10,1.41,17,.370
3,1170,233,.219,.268,106,2.98,5,38,9,0,...,1,0,0,1306,1,2,18,1.13,7,.474
4,3222,762,.277,.311,276,3.05,2,104,49,1,...,1,2,5,3589,5,22,35,1.43,31,.398


In [87]:
career_pitching_df = career_pitching_df[
    [
        'player_id',
        'era',
        'ip',
        'avg',
        'h',
        'h9',
        'hr',
        'hr9',
        'bb',
        'bb9',
        'so',
        'k9',
        'kbb',
    ]
]

career_pitching_df.rename(columns = {
    'era': 'career_era',
    'ip': 'career_innings_pitched',
    'avg': 'career_oppose_bat_avg',
    'h': 'career_hits',
    'h9': 'career_hits_per_9',
    'hr': 'career_homeruns',
    'hr9': 'career_homeruns_per_9',
    'bb': 'career_walks',
    'bb9': 'career_walks_per_9',
    'so': 'career_strikeouts',
    'k9': 'career_strikeouts_per_9',
    'kbb': 'career_strikeouts_to_walks'
}, inplace=True)

career_pitching_df

,player_id,career_era,career_innings_pitched,career_oppose_bat_avg,career_hits,career_hits_per_9,career_homeruns,career_homeruns_per_9,career_walks,career_walks_per_9,career_strikeouts,career_strikeouts_per_9,career_strikeouts_to_walks
0,605200,3.78,507.1,.263,509,9.03,54,0.96,145,2.57,368,6.53,2.54
1,547943,3.07,610.0,.248,573,8.45,61,0.90,143,2.11,556,8.20,3.89
2,607192,4.66,245.1,.245,227,8.33,33,1.21,119,4.37,271,9.94,2.28
3,622491,3.51,320.2,.219,256,7.19,42,1.18,106,2.98,339,9.51,3.20
4,543475,5.09,814.1,.277,892,9.86,101,1.12,276,3.05,597,6.60,2.16


In [88]:
career_pitching_df_merge = career_pitching_df.merge(pitching_leader_df, on = 'player_id')
career_pitching_df_merge

,player_id,career_era,career_innings_pitched,career_oppose_bat_avg,career_hits,career_hits_per_9,career_homeruns,career_homeruns_per_9,career_walks,career_walks_per_9,...,innings_pitched,opposing_bat_avg,hits,hits_per_9,homeruns,walks,walks_per_9,strikeouts,strikeouts_per_9,strikeouts_to_walks
0,605200,3.78,507.1,.263,509,9.03,54,0.96,145,2.57,...,52.2,.240,46,7.86,4,16,2.73,36,6.15,2.25
1,547943,3.07,610.0,.248,573,8.45,61,0.90,143,2.11,...,52.1,.189,35,6.02,6,3,0.52,54,9.29,18.00
2,607192,4.66,245.1,.245,227,8.33,33,1.21,119,4.37,...,48.1,.202,35,6.52,3,9,1.68,55,10.24,6.11
3,622491,3.51,320.2,.219,256,7.19,42,1.18,106,2.98,...,61.2,.163,34,4.96,3,25,3.65,76,11.09,3.04
4,543475,5.09,814.1,.277,892,9.86,101,1.12,276,3.05,...,45.2,.199,32,6.31,4,16,3.15,47,9.26,2.94


In [89]:
career_pitching_df_merge = career_pitching_df_merge[
    [
        'player_id',
        'name',
        'career_era',
        'career_innings_pitched',
        'career_oppose_bat_avg',
        'career_hits',
        'career_hits_per_9',
        'career_homeruns',
        'career_homeruns_per_9',
        'career_walks',
        'career_walks_per_9',
        'career_strikeouts',
        'career_strikeouts_per_9',
        'career_strikeouts_to_walks'
    ]
]
career_pitching_df_merge

,player_id,name,career_era,career_innings_pitched,career_oppose_bat_avg,career_hits,career_hits_per_9,career_homeruns,career_homeruns_per_9,career_walks,career_walks_per_9,career_strikeouts,career_strikeouts_per_9,career_strikeouts_to_walks
0,605200,"Davies, Zach",3.78,507.1,.263,509,9.03,54,0.96,145,2.57,368,6.53,2.54
1,547943,"Ryu, Hyun-Jin",3.07,610.0,.248,573,8.45,61,0.90,143,2.11,556,8.20,3.89
2,607192,"Glasnow, Tyler",4.66,245.1,.245,227,8.33,33,1.21,119,4.37,271,9.94,2.28
3,622491,"Castillo, Luis",3.51,320.2,.219,256,7.19,42,1.18,106,2.98,339,9.51,3.20
4,543475,"Lyles, Jordan",5.09,814.1,.277,892,9.86,101,1.12,276,3.05,597,6.60,2.16


In [90]:
career_pitching_dict = career_pitching_df_merge.to_dict(orient='records')
pprint(career_pitching_dict)

[{'career_era': '3.78',
  'career_hits': '509',
  'career_hits_per_9': '9.03',
  'career_homeruns': '54',
  'career_homeruns_per_9': '0.96',
  'career_innings_pitched': '507.1',
  'career_oppose_bat_avg': '.263',
  'career_strikeouts': '368',
  'career_strikeouts_per_9': '6.53',
  'career_strikeouts_to_walks': '2.54',
  'career_walks': '145',
  'career_walks_per_9': '2.57',
  'name': 'Davies, Zach',
  'player_id': '605200'},
 {'career_era': '3.07',
  'career_hits': '573',
  'career_hits_per_9': '8.45',
  'career_homeruns': '61',
  'career_homeruns_per_9': '0.90',
  'career_innings_pitched': '610.0',
  'career_oppose_bat_avg': '.248',
  'career_strikeouts': '556',
  'career_strikeouts_per_9': '8.20',
  'career_strikeouts_to_walks': '3.89',
  'career_walks': '143',
  'career_walks_per_9': '2.11',
  'name': 'Ryu, Hyun-Jin',
  'player_id': '547943'},
 {'career_era': '4.66',
  'career_hits': '227',
  'career_hits_per_9': '8.33',
  'career_homeruns': '33',
  'career_homeruns_per_9': '1.21',


---

### MongoDB (with pymongo)

In this section, we will take all of the previously-generated dictionaries, lists, etc., and load them to a Mongo database called mlbDB.

In [91]:
# establish Mongo connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [92]:
# For purposes of this project, we will drop the mlbDB database first so that we
# do not duplicate collections, documents, etc.
client.drop_database('mlbDB')

In [93]:
# Create mlbDB database and first collection, teams
db = client.mlbDB
collection_one = db.teams

In [94]:
# Insert teams_dict and rox_roster_dict from first two sections into teams collection
# collection_one.insert_many([
#     teams_dict#,
#     #rox_roster_dict    
# ])

#Switched to adding Rockies' roster to Rockies collection, as it doesn't really make sense in teams collection
#without any way to "know" it's the rockies players listed
collection_one.insert_many([teams_dict])
db.rockies.insert_many([rox_roster_dict])

In [95]:
# Follow methodology above to create the collections top_hitters, top_hitters_career,
# top_pitchers, and top_pitchers_career, and insert appropriate distionaries (or lists) into these collections
collection_two = db.top_hitters
collection_two.insert_many(hitting_leader_dict)

In [96]:
collection_three = db.top_hitters_career
collection_three.insert_many(career_hitting_dict)

In [97]:
collection_four = db.top_pitchers
collection_four.insert_many(pitching_leader_dict)

In [98]:
collection_five = db.top_pitchers_career
collection_five.insert_many(career_pitching_dict)

---

## MLB Official Website Scraping

---

In [99]:
#defining variables
executable_path = {'executable_path': 'chromedriver.exe'}
#browser = Browser('chrome', **executable_path, headless=False)

#already definned right above
# conn = 'mongodb://localhost:27017'
# client = pymongo.MongoClient(conn)
# db = client.mlbDB

---

### Scrape top headlines from mlb.com

Reading the headlines "box" on the homepage for MLB. After pulling the articles, I parse the article text to find the player names and their player_ids, which could be used in the API above.

In [100]:
get_url = 'https://www.mlb.com/'
html = requests.get(get_url)
soup = BeautifulSoup(html.text, 'html.parser')
top_headlines = []
results = soup.select('div.l-grid div.p-headline-stack')
#there are three div.p-headline-stack s, but index 0 and 2 are hidden ones that don't contain the same articles.
#index 1 is for xs and sm, index 3 is for md, lg, xl and I used index 3. code works the same with index 1.
for result in results[3].select('li.p-headline-stack__headline'):
    if result.a['href'][:24] == 'https://www.mlb.com/news':
        #some news articles link to mlb.com/cut4 which has a completely different layout for the next part
        #so i just dropped those articles
        top_headlines.append({'headline':result.text, 'headline_url':result.a['href']})
#print(top_headlines)
browser = Browser('chrome', **executable_path, headless=False)
for i, each_article in enumerate(top_headlines):
    get_url = each_article['headline_url']
#     html = requests.get(get_url)
#     soup = BeautifulSoup(html.text, 'html.parser')

# had to use chromedriver instead because for some articles, the player names were linked in <a href> but others,
# the request would get <forge-entity> that would be converted to <a> after it had already 'requests.get'-ed
    
    browser.visit(get_url)

    while True:
        html = browser.html
        soup = BeautifulSoup(html,'html.parser')
        try:
            results = soup.select('div.template-article__content div.article-item__body')
            break
        except:
            time.sleep(0.5)
    
    article_info = []
    #within the body of the first (index=0) article (because this page will load multiple articles)
    articlebody_results = results[0].select('p a')
    #find any link
    for each_link in articlebody_results:
        #and if the article links to a player page...
        if each_link['href'][:27] == 'https://www.mlb.com/player/':
            #add their name and their player id (last 6 characters of url)
            #seems mlb.com content writers are good about listing full name when they link a player
            #so going to the page and pulling their name or using the API to pull their name is unnecessary
            #some articles don't properly link a player to their player page on first mention, but oh well.
            article_info.append({'player_name':each_link.text, 'player_id':each_link['href'][-6:]})
    #print(article_info)
    top_headlines[i]['players_in_article'] = article_info
    #needed a counter, i, to correctly add the name-list to the correct article
    #i += 1
browser.quit()
print(top_headlines)
db.top_headlines.insert_many(top_headlines)

[{'headline': 'Yanks score 3 in bottom of 9th to take over 1st', 'headline_url': 'https://www.mlb.com/news/yankees-walk-off-to-beat-rays', 'players_in_article': [{'player_name': 'Gio Urshela', 'player_id': '570482'}, {'player_name': 'CC Sabathia', 'player_id': '282332'}, {'player_name': 'Kendrys Morales', 'player_id': '434778'}, {'player_name': 'Luke Voit', 'player_id': '572228'}, {'player_name': 'Gary Sánchez', 'player_id': '596142'}, {'player_name': 'Gleyber Torres', 'player_id': '650402'}, {'player_name': 'Thairo Estrada', 'player_id': '642731'}, {'player_name': 'Clint Frazier', 'player_id': '640449'}, {'player_name': 'Cameron Maybin', 'player_id': '457727'}]}, {'headline': '\r3 homers, 3 innings: Historic night for Kris Bryant', 'headline_url': 'https://www.mlb.com/news/kris-bryant-hits-three-home-runs', 'players_in_article': [{'player_name': 'Kris Bryant', 'player_id': '592178'}, {'player_name': 'Max Scherzer', 'player_id': '453286'}, {'player_name': 'Kyle Schwarber', 'player_id':

---

### top Rockies headline, URL, referenced player names and IDs

Running essentially the same code, but for the team website instead of mlb.com.

In [101]:
get_url = 'https://www.mlb.com/rockies'
html = requests.get(get_url)
soup = BeautifulSoup(html.text, 'html.parser')
#top_cr_headline = []
results = soup.select('div.l-grid div.p-headline-stack')
top_result = results[3].select('li.p-headline-stack__headline')[0]
top_cr_headline = {'headline':top_result.text.strip(), 'headline_url':top_result.a['href']}
#print(top_cr_headline)

browser = Browser('chrome', **executable_path, headless=False)
get_url = top_cr_headline['headline_url']
#     html = requests.get(get_url)
#     soup = BeautifulSoup(html.text, 'html.parser')
# had to use chromedriver because for some articles, the player names were linked in <a href> but others,
# the request would get <forge-entity> that would be converted to <a> after it had already requests.get-ed

browser.visit(get_url)

while True:
    html = browser.html
    soup = BeautifulSoup(html,'html.parser')
    try: 
        results = soup.select('div.template-article__content div.article-item__body')
        break
    except:
        time.sleep(0.5)

article_info = []
articlebody_results = results[0].select('p a')
#[0] because only pull the first article
for each_name in articlebody_results:
    if each_name['href'][:27] == 'https://www.mlb.com/player/':
        article_info.append({'player_name':each_name.text, 'player_id':each_name['href'][-6:]})
top_cr_headline['players_in_article'] = article_info
browser.quit()
print(top_cr_headline)
db.rockies.insert_one(top_cr_headline)

{'headline': "Gray fans eight but can't escape fifth in loss", 'headline_url': 'https://www.mlb.com/rockies/news/jon-gray-strikes-out-8-in-loss-to-phillies', 'players_in_article': [{'player_name': 'Jon Gray', 'player_id': '592351'}, {'player_name': 'Ian Desmond', 'player_id': '435622'}, {'player_name': 'Carlos Estevez', 'player_id': '608032'}]}


---

### most recent rockies video

Use splinter to pull search page for mlb's video page, then pull the first result and visit that page and
pull the direct link to the video file

In [102]:
#hard coded Rockies team ID
cr_team_id = 115
video_search_url = 'https://www.mlb.com/video/search/tag/teamid-'+str(cr_team_id)
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(video_search_url)

while True:
    html = browser.html
    soup = BeautifulSoup(html,'html.parser')
    try:
        results = soup.select('div.video-preview')
        break
    except:
        time.sleep(0.5)

video_page_url = 'https://www.mlb.com' + results[0].select('a')[0]['href']
browser.visit(video_page_url)

while True:
    html = browser.html
    soup2 = BeautifulSoup(html,'html.parser')
    try:
        video_wrapper = soup2.select('div#video-section__wrapper meta')
        break
    except:
        time.sleep(0.5)

video_result = {}

for each_meta in video_wrapper:
    if each_meta['itemprop'] == 'name':
        video_result['name'] = each_meta['content']
    elif each_meta['itemprop'] == "contentURL":
        video_result['url'] = each_meta['content']
    elif each_meta['itemprop'] == "description":
        video_result['desc'] = each_meta['content']
print(video_result)
browser.quit()

db.rockies.insert_one(video_result)

{'name': 'CG: COL@PHI - 5/17/19', 'desc': 'Condensed Game: COL@PHI - 5/17/19', 'url': 'https://cuts.diamond.mlb.com/FORGE/2019/2019-05/17/f05df37d-fc1eccde-76b80fa0-csvm-diamondx64-asset_1280x720_59_4000K.mp4'}


---

### Most recent videos featuring the top hitting leaders and top pitching leaders

Use the player IDs generated via the API above, read from DB, and run a search on MLB.com's video page to find their most recent video

In [103]:
#top hitters and top pitchers are two different collections, so setting a list to loop through:
top_p = [db.top_hitters,db.top_pitchers]

#open browser
browser = Browser('chrome', **executable_path, headless=False)

#loop through the list of two collections
for each_cat in top_p:
    #within each collection, loop through each player in the collection
    for each_top_p in each_cat.find():
        player_id = str(int(each_top_p['player_id']))
        #all player_ids should be strings, but convert to int to str should fix any issues with
        #player_id being stored as a double (which leads to ######.# format)
        video_search_url = 'https://www.mlb.com/video/search/tag/playerid-'+player_id

        browser.visit(video_search_url)

        while True:
            html = browser.html
            soup = BeautifulSoup(html,'html.parser')
            try:
                results = soup.select('div.video-preview')
                break
            except:
                time.sleep(0.5)

        video_page_url = 'https://www.mlb.com' + results[0].select('a')[0]['href']

        browser.visit(video_page_url)

        while True:
            html = browser.html
            soup2 = BeautifulSoup(html,'html.parser')
            try:
                video_wrapper = soup2.select('div#video-section__wrapper meta')
                break
            except:
                time.sleep(0.5)

        video_result = {}

        for each_meta in video_wrapper:
            if each_meta['itemprop'] == 'name':
                video_result['vid_name'] = each_meta['content']
            elif each_meta['itemprop'] == "contentURL":
                video_result['vid_url'] = each_meta['content']
            elif each_meta['itemprop'] == "description":
                video_result['vid_desc'] = each_meta['content']
        #print(video_result)
        each_cat.update_one({'player_id':each_top_p['player_id']},{'$set':video_result})
browser.quit()

---

### Pulling standings from mlb.com/standings

Use pandas read_html function to read the standings page directly to a pandas dataframe then write to mongodb

In [ ]:
browser = Browser('chrome', **executable_path, headless=False)
browser.visit('https://www.mlb.com/standings')

#wait for tables to load
while True:
    html = browser.html
    try:
        tables_list = pd.read_html(html)
        break
    except:
        time.sleep(0.5)

browser.quit()

all_divisions = []

#for some reason each division is displayed twice, though only one of each is visible.
for i in range(0,11,2):
    all_divisions.append(tables_list[i])

for each_division in all_divisions:
    each_division.rename(columns={'>.500':'over500'},inplace=True)
    #rename column as the period breaks
    data = each_division.to_dict(orient='records')
    db.standings.insert_many(data)

In [ ]:
db.teams_split.insert_many(teams_df.reset_index().to_dict(orient="records"))

In [47]:
# teams_df_test = teams_df.reset_index()
#teams_df_test.at[19,'team_name'] = 'St Louis Cardinals'
#rownum = teams_df_test.loc[teams_df_test['team_name'] == 'St. Louis Cardinals'].index
#teams_df_test.at[rownum,'team_name'] = 'St Louis Cardinals'
# teams_df_test

In [ ]:
db.rockies_roster.insert_many(rox_roster_df.reset_index().to_dict(orient="records"))